In [52]:
from dotenv import load_dotenv
from pprint import pprint
# Load environment variables
load_dotenv()

True

In [53]:
import logging
import os

log = logging.getLogger(__name__)


def get_user_agent() -> str:
    """Get user agent from environment variable."""
    env_user_agent = os.environ.get("USER_AGENT")
    if not env_user_agent:
        log.warning(
            "USER_AGENT environment variable not set, "
            "consider setting it to identify your requests."
        )
        return "DefaultLangchainUserAgent"
    return env_user_agent

get_user_agent()

USER_AGENT environment variable not set, consider setting it to identify your requests.


'DefaultLangchainUserAgent'

In [75]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.2-90b-vision-preview")


In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.tools import StructuredTool, ToolException
from typing import List
from bs4 import BeautifulSoup
import requests


# Tavily search
tavily_search_tool = TavilySearchResults(max_results=2)

class WebScraper:
    def __init__(self, urls: List[str]):
        self.urls = urls

    def scrape_webpages(self) -> str:
        """Use requests and bs4 to scrape the provided web pages for detailed information."""
        try:
            loader = WebBaseLoader(self.urls)
            docs = loader.load()
            return "\n\n".join(
                [
                    f'<Document name="{doc.metadata.get("title", "")}">\n{doc.page_content}\n</Document>'
                    for doc in docs
                ]
            )
        except ToolException as e:
            return self._handle_error(e)

    def _handle_error(self, error: ToolException) -> str:
        return f"The following errors occurred during tool execution: `{error.args[0]}`"

web_scraper_tool = StructuredTool.from_function(
    func=WebScraper.scrape_webpages,
    handle_tool_error=WebScraper._handle_error
)


from langchain_experimental.utilities import PythonREPL
import nbformat

repl = PythonREPL()

def python_repl(
    code: Annotated[str, "The Python code to execute to generate visualization."],
):
    """Use this to execute python code."""
    try:

        result = repl.run(code)
    except ToolException as e:
        return f"Failed to execute. Error: {repr(e)}"
    return f"Successfully executed:\n```python\n{code}\n```\nStdout: {result}"


repl_tool = StructuredTool.from_function(
    func=python_repl,
)


def extract_code_from_response(response: str) -> str:
    start = response.find('```python') + len('```python')
    end = response.find('```', start)
    return response[start:end].strip()


from langgraph.prebuilt import ToolNode
tools = [tavily_search_tool, web_scraper_tool, repl_tool]
tool_node = ToolNode(tools)

In [76]:
llm_with_tools = llm.bind_tools(tools)

In [78]:
llm_with_tools.invoke('Generate a pie chart with random values')

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_vwwn', 'function': {'arguments': '{"code": "import matplotlib.pyplot as plt\\nimport random\\n\\nlabels = [\'A\', \'B\', \'C\', \'D\', \'E\']\\nvalues = [random.randint(1, 100) for _ in range(5)]\\n\\nplt.pie(values, labels=labels)\\nplt.show()"}', 'name': 'python_repl'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 77, 'prompt_tokens': 835, 'total_tokens': 912, 'completion_time': 0.449477365, 'prompt_time': 0.074625543, 'queue_time': -0.471637918, 'total_time': 0.524102908}, 'model_name': 'llama-3.2-90b-vision-preview', 'system_fingerprint': 'fp_9c2a937c92', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4cc2f669-188b-4f28-8704-c3189477306b-0', tool_calls=[{'name': 'python_repl', 'args': {'code': "import matplotlib.pyplot as plt\nimport random\n\nlabels = ['A', 'B', 'C', 'D', 'E']\nvalues = [random.randint(1, 100) for _ in range(5)]\n\nplt.pie(values, labels=labels)\nplt.s

In [64]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)
graph = graph_builder.compile()
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [("user", user_input)]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)


while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

Assistant: It seems like you didn't ask a question or provide any information. If you need assistance, feel free to ask, and I'll do my best to help.
Goodbye!
